
<img src="https://static.tildacdn.com/tild3762-6664-4936-a364-663135373331/_1.png" width="240" height="240" align="center"/>
 
<h1><center>Frequently asked question answering with DeepPavlov</center></h1>

Лаборатория нейронных сетей и глубокого обучения МФТИ работает над библиотекой [DeepPavlov](https://github.com/deepmipt/DeepPavlov) — библиотека для создания диалоговых систем. Она содержит набор претренированных компонент для анализа языка, с помощью которых можно эффективно решать задачи бизнеса.

Одна из важных задач для бизнеса это организация ответов на часто задаваемые вопросы клиентов. Сделать это возможно через колл-центр, или виджет на сайте, наняв сотрудников специально для этого.

В этом ноутбуке я хочу показать как использовать модели, которые отвечают на часто задаваемые вопросы клиентов. В рамках **DeepPavlov** эти модели называются autoF.A.Q. В качеcтве датасета, я буду использовать датасет вопросов и ответов лицеистов МФТИ.

## Установка

Для работы с библиотекой установите **Python 3.6**, активируйте среду разработки. Затем установите **DeepPavlov**.

In [ ]:
!pip install -q deeppavlov

## Датасет

В качестве датасета выступает файл формата **csv**, с заголовками **Question** и **Answer**. Пример вопросов и ответов ниже.

In [7]:
import pandas as pd
df = pd.read_csv("http://files.deeppavlov.ai/faq/school/faq_school.csv")
print(df.sample(frac=1)[:5].to_string())

                                             Question                                             Answer
12                        Как войти в личный кабинет?  Подробно про личный кабинет вы можете прочитат...
8   Я ошиблась в имени ребенка. Где можно его изме...  Некоторые данные можно поменять через личный к...
40  Как проходит первый этап вступительных испытан...  Вступительные испытания на первом этапе предст...
42       Какие экзамены надо сдавать в девятый класс?  Вступительные испытания на первом этапе предст...
13                      Как попасть в личный кабинет?  Подробно про личный кабинет вы можете прочитат...


## Описание моделей

На вход модели подается вопрос, далее модель определяет наиболее близкий нашему вопрос из датасета, и возвращает соответствующий ответ. **DeepPavlov** содержит несколько моделей, которые решают проблему поиска ответов на популярные вопросы. Подробную информацию по деталям реализации моделей вы можно найти в нашей [справке](http://docs.deeppavlov.ai/en/master/skills/faq.html). Следующая команда показывает нам список моделей autoF.A.Q.

In [1]:
from IPython.display import display
from deeppavlov import configs
display(configs.faq.keys())

frozenset({'fasttext_avg_autofaq',
           'fasttext_tfidf_autofaq',
           'tfidf_autofaq',
           'tfidf_logreg_autofaq',
           'tfidf_logreg_en_faq'})

## Работа с моделями через **командную строку**

Режим работы через командую строку позволяет установить зависимости модели (с помощью команды **install**), натренировать модель на основе датасета (**train**), и взаимодействовать с моделью (**interact**). Ниже приведены примеры всех этих команд.

In [ ]:
# установка пакетов и файлов, требуемых для работы модели
!python -m deeppavlov install tfidf_autofaq
# тренировка модели
!python -m deeppavlov train tfidf_autofaq
# взаимодействие с моделью
!python -m deeppavlov interact tfidf_autofaq

## Работа с моделями через **Python**

Для того чтобы работать с моделями через **Python** необходимо импортировать файл конфигурации и метод **build_model**. Далее построить модель с помощью метода **build_model**. На вход модель принимает массив с текстовыми запросами, в качестве выхода модель возвращает соответствующие ответы с оценкой уверенности.

In [3]:
from deeppavlov import configs, build_model
faq = build_model(configs.faq.tfidf_autofaq, download = True)
faq(['мне нужен код регистрации'])

[['Вы можете написать нам письмо на почту приемной кампании, с адреса, который указали при регистрации или обратиться к нашим сотрудниками через Telegram и предоставить информацию, которая позволит идентифицировать вашего ребенка (ФИО, дата рождения и адрес электронной почты, указанный при регистрации).'],
 [0.3]]

## Скилл **SimilarityMatchingSkill**

Кроме того библиотека **DeepPavlov** содержит в себе класс **SimilarityMatchingSkill** - упрощенный интерфейс работы с моделями autoF.A.Q. Чтобы продемонстрировать его работу, создайте объект класса **SimilarityMatchingSkill**, определив следующие параметры.

* **data_path** - путь к csv файлу с данными
* **x_col_name** - имя колонки с вопросами в csv файле (Question, по умолчанию)
* **y_col_name** - имя колонки с ответами в csv файле (Answer, по умолчанию)
* **edit_dict** - dict с параметрами конфигурации для перезаписи
* **save_load_path** - путь куда сохранить натренированную модель
* **train** - тренировать ли модель?

In [5]:
from deeppavlov.contrib.skills.similarity_matching_skill import SimilarityMatchingSkill

faq_skill = SimilarityMatchingSkill(data_path = 'http://files.deeppavlov.ai/faq/school/faq_school.csv',
                              x_col_name = 'Question', 
                              y_col_name = 'Answer',
                              save_load_path = './model',
                              config_type = 'tfidf_autofaq',
                              train = False)

Обратите внимание, что навыки (skills) в **DeepPavlov** в отличие от компонент имеют унифицированный вход, где первый параметр это массив текстовых обращений, второй параметр массив истории общений, третий параметр массив состояний. В навыке нашего класса обязательным является лишь первый параметр - массив текстовых обращений. Убедитесь в то, что навык корректо отвечает на запросы.

In [6]:
faq_skill(['что такое вступительные экзамены'], [], [])

(['Вступительные испытания на первом этапе представляют собой выполнение письменных работ по профильным предметам.'],
 [0.2])

## Интеграция моделей в Яндекс.Алиса

Для того чтобы интегрировать нашу модель в навык Яндекс.Алиса. Создадим объект класса **DefaultAgent**.

In [7]:
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector

agent = DefaultAgent([faq_skill], skills_selector=HighestConfidenceSelector())

Далее запустим сервер с указанием пути для запросов **endpoint='faq'** и порта подключения **port=5000**

In [ ]:
from deeppavlov.utils.alice import start_agent_server

start_agent_server(agent, host='0.0.0.0', port=5000, endpoint='/faq')

 * Serving Flask app "deeppavlov.utils.alice.alice" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


Обратите внимание, что Яндекс.Диалоги в качестве **Webhook URL** требует указывать сервер с внешним IP адресом и доступом по протоколу https. Для быстрого прототипирования вы можете использовать [ngrok](https://ngrok.com/). Ngrok позволит вам создавать туннель для доступа к вашему серверу с **DeepPavlov** в локальной сети, для этого запустите **ngrok http 5000** на вашем сервере с **DeepPavlov**. В ответ на это будет создано два туннеля, по одному на протоколы **http** и **https**. Скопируйте адрес туннеля для **https**, добавьте к линку эндпоинт **/faq**, итоговый линк будет **Webhook URL** для нашего Яндекс.Диалога. Далее заполните поля необходимые для сохранения черновика диалога. Сохраните черновик и перейдите на вкладку Тестирование.

## Ссылки

### [DeepPavlov documentation](http://docs.deeppavlov.ai/en/master/skills/faq.html)

### [DeepPavlov Blog](https://medium.com/deeppavlov)

### [Forum](https://forum.ipavlov.ai)

### [GitHub Repository](https://github.com/deepmipt/DeepPavlov)

### [DeepPavlov demo page](https://demo.ipavlov.ai)